<a href="https://colab.research.google.com/github/pawanbhatta178/FakeReviewDetector/blob/main/ReviewAIDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ### Problem Statement

User generated reviews for products has been a cornerstone for people’s interest in purchase.
However, the rise of the Large Language Model (LLM) has made generating these reviews
inexpensive, quick and trivial. This means the usage of LLM to generate fake reviews is on the
rise. The goal of this project is to build and explore different strategies for the classification
problems at hand: Statistical (Simple), Supervised Transformer based, and Self detection using
LLM. First one is based on statistical differences of the text generated by LLM as compared to
humans, with respect to attributes like token distribution and entropy and perplexity. This is
based on the notion that human generated text usually has higher perplexity than AI generated.
We plan to also experiment with the second approach where we fine tune transformer based
models to do the classification task. We compare the performances of above-mentioned
approaches. For the final strategy, we prompt LLM to self detect if the given text is generated by
AI or not.


### Data acquisition

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Dataset 1:  [Ai vs Human](https://www.kaggle.com/datasets/shanegerami/ai-vs-human-text?resource=download)

This dataset consist of laballed human and AI generated essays.

##### Getting the dataset

Make sure to download the ai vs human  zip file from [kaggle](https://www.kaggle.com/datasets/shanegerami/ai-vs-human-text?resource=download), unzip it and upload a copy of AI_Human csv in google drive. Make sure to create the exact directory structure as in `csv_path` so all contributors can work with same version of this code.

In [6]:
import pandas as pd

csv_path = '/content/drive/MyDrive/Grad/NLP/FinalProject/AI_Human.csv'
df = pd.read_csv(csv_path)

df.info()     # column names + dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 487235 entries, 0 to 487234
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   text       487235 non-null  object 
 1   generated  487235 non-null  float64
dtypes: float64(1), object(1)
memory usage: 7.4+ MB


##### AI Generated Texts

In [8]:
df_ai=df[df['generated']==1]
df_ai.info()
# df_ai.head()


<class 'pandas.core.frame.DataFrame'>
Index: 181438 entries, 704 to 487232
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   text       181438 non-null  object 
 1   generated  181438 non-null  float64
dtypes: float64(1), object(1)
memory usage: 4.2+ MB


In [9]:
df_ai.head(10)

,text,generated
704,"This essay will analyze, discuss and prove one...",1.0
740,I strongly believe that the Electoral College ...,1.0
1262,"Limiting car use causes pollution, increases c...",1.0
1378,Car-free cities have become a subject of incre...,1.0
1379,"Car Free Cities Car-free cities, a concept ga...",1.0
1380,A Sustainable Urban Future Car-free cities ...,1.0
1381,Pioneering Sustainable Urban Living In an e...,1.0
1382,The Path to Sustainable Urban Living In an ...,1.0
1383,A Paradigm Shift in Urban Living In an era ...,1.0
1384,Revolutionizing Urban Living In an age defi...,1.0


##### Human Generated Texts

In [10]:
df_human=df[df['generated']==0]
df_human.info()

<class 'pandas.core.frame.DataFrame'>
Index: 305797 entries, 0 to 487234
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   text       305797 non-null  object 
 1   generated  305797 non-null  float64
dtypes: float64(1), object(1)
memory usage: 7.0+ MB


In [11]:
df_human.head(10)

,text,generated
0,Cars. Cars have been around since they became ...,0.0
1,Transportation is a large necessity in most co...,0.0
2,"""America's love affair with it's vehicles seem...",0.0
3,How often do you ride in a car? Do you drive a...,0.0
4,Cars are a wonderful thing. They are perhaps o...,0.0
5,The electrol college system is an unfair syste...,0.0
6,"Dear state senator, It is the utmost respect t...",0.0
7,"Fellow citizens, cars have become a major role...",0.0
8,"""It's official: The electoral college is unfai...",0.0
9,The Electoral College has been kept for centur...,0.0


### Installing Libraries for Model Pretraining

In [15]:
!pip install -q datasets
!pip install -q transformers



### GPT 2 Model

We plan to run GPT model and calculated perplexity on dataset. Analyze if indeed perplexity is different for human vs AI generated text.

In [26]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load model and tokenizer
model_name = "gpt2"  # or "gpt2-medium", "gpt2-large", "gpt2-xl"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Move model to GPU if available
device_type="cuda" if torch.cuda.is_available() else "cpu"
print(device_type)
device = torch.device(device_type)
model = model.to(device)


cuda


In [17]:
prompt = "Mount everest is located in  "
input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

# Generate text
output = model.generate(
    input_ids,
    max_length=100,
    num_return_sequences=1,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.9,
    pad_token_id=tokenizer.eos_token_id,
)

# Decode and print
print(tokenizer.decode(output[0], skip_special_tokens=True))


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Mount everest is located in   the town of Kastia, where the townspeople are known as huts of the gods. There are many huts on the other side of the lake that make you wonder, "Where is God? Why is there such a thing as a 'hut of the gods?' I'd like to ask you these questions. God, I'm curious what does this mean? Why does it have to be this way? Do you believe in a


In [18]:
test_sentence = "Hello good afternoon"


inputs = tokenizer(test_sentence, return_tensors="pt")
input_ids = inputs["input_ids"].to(device)
attention_mask = inputs["attention_mask"].to(device)


with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
    # loss is already the average negative log-likelihood
    neg_log_likelihood = outputs.loss


perplexity = torch.exp(neg_log_likelihood)
print(f"Perplexity: {perplexity.item():.2f}")


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Perplexity: 504.90


### BERT Model

In [28]:
# Import libraries
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

In [29]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=128
    )


In [21]:
# Stratified 20k sample
df_sampled, _ = train_test_split(
    df,
    train_size=30000,
    stratify=df['generated'],
    random_state=42
)

# Confirm class balance
print(df_sampled['generated'].value_counts(normalize=True))


generated
0.0    0.627633
1.0    0.372367
Name: proportion, dtype: float64


In [30]:
# 80/20 split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_sampled['text'], df_sampled['generated'],
    test_size=0.2,
    random_state=42
)


In [31]:
# Convert to Hugging Face datasets
train_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels})
val_dataset = Dataset.from_dict({'text': val_texts, 'label': val_labels})

# Tokenize with multiprocessing
train_dataset = train_dataset.map(tokenize_function, batched=True, num_proc=4)
val_dataset = val_dataset.map(tokenize_function, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/24000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/6000 [00:00<?, ? examples/s]

In [32]:
# Load pre-trained BERT model with classification head
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    save_strategy='epoch',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10,
    load_best_model_at_end=True
)
# Set up Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
trainer.train()


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pawanbhatta178 (pawanbhatta178-asdsd) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


ValueError: Target size (torch.Size([16])) must be the same as input size (torch.Size([16, 2]))